In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [7]:
# Problem parameters
sigma_s = 0 # Macroscopic cross section for scattering [cm^(-1)] {Play with sigma_s = ln(2) or something}
sigma_t = 1 # Macroscopic total interaction cross section [cm^(-1)] (reciprocal is neutron mean free path (mfp))
q_ext = 3 # External source density rate [TODO: insert units]
# q = q_ext/2.0 # Source density rate [TODO: insert units]

# Left- and right-endpoints of domain
a = 0
b = 10

# Number of cells in domain
ncells = 9
# Number of points in domain
npts   = ncells + 1
points = np.linspace(a, b, npts)
dx     = np.abs(points[1] - points[0])

# Auxillary term for calculation clarity in the loops below
aux_term = (sigma_t/2.0)*dx

# BCs
psi_left = 0.5
psi_right = 1.5

# Quadrature order (needs to be EVEN to avoid mu = 0)
m = 16
angles, weights = scipy.special.roots_legendre(m)
# By default, angles go from negative to positive, and we want the opposite
angles = np.flip(angles)

# Array to store angular fluxes in all directions at the previous cell interface
psi_prev = np.full((m, ), psi_right) # Enforce right BC
psi_prev[0::m//2] = psi_left # Enforce left BC

# Array to store scalar fluxes at the midpoint of each cell
sca_fluxes = np.full((ncells, ), np.log(2)) # Initial guesses for midpoint scalar fluxes

# If the absolute difference of successive iterates of the scalar flux at
# cell interface i is below this, then we stop updating the scalar flux at 
# that interface
eps = 1e-14

for i in np.arange(0, ncells):
    next_sca_flux = 0.0
    sca_flux_err = np.abs(sca_fluxes[i] - next_sca_flux)
    
    # Initial guess for the source term at midpoint of cell i
    curr_source = (q_ext/2.0) + (sigma_s/2.0)*sca_fluxes[i]

    while sca_flux_err >= eps: 
        for j, mu in enumerate(angles):
            abs_mu = np.abs(mu) 
            psi_next_i_j = ((abs_mu - aux_term)*psi_prev[j] + curr_source*dx) / \
                            (abs_mu + aux_term)

            # Update scalar flux at cell interface i using the average of the
            # angular flux at the previous cell interace and the just computed angular
            # flux at the current cell interface 
            psi_avg_i_j            = (psi_next_i_j + psi_prev[j])/2.0
            next_sca_flux   += psi_avg_i_j * weights[j]

            # If the difference is small enough, set the angular flux at the 
            # previous cell interface to be the angular flux at the current interface
            if sca_flux_err < eps:
                psi_prev[j] = psi_next_i_j

        sca_flux_err = np.abs(next_sca_flux - sca_fluxes[i])

        # Update the source term
        sca_fluxes[i] = next_sca_flux
        curr_source    = (q_ext/2.0) + (sigma_s/2.0)*sca_fluxes[i]
        next_sca_flux  = 0.0

In [8]:
print(sca_fluxes)

[2.56933648 2.56933648 2.56933648 2.56933648 2.56933648 2.56933648
 2.56933648 2.56933648 2.56933648]


In [ ]:
sca_fluxes = np.zeros((ncells, ))

psi_prev_l = psi_left
psi_prev_r = psi_right 

for d, mu in enumerate(angles):
    abs_mu = np.abs(mu)
    init_sca_flux_d = np.log(2)
    next_sca_flux_d = 0.0

    sca_flux_err_d = np.abs(init_sca_flux_d - next_sca_flux_d)

    while sca_flux_err >= eps:
        for i in np.arange(ncells):
            curr_source = (q_ext + sigma_s * init_sca_flux_d)/2.0
            # Compute angular flux at current direction
            if mu > 0:
                psi_next = ((abs_mu - aux_term)*psi_prev_l + curr_source*dx) / \
                            (abs_mu + aux_term)
                # Compute midpoint angular flux
                avg_psi = (psi_next + psi_prev_l)/2.0
            else:
                psi_next = ((abs_mu - aux_term)*psi_prev_r + curr_source*dx) / \
                            (abs_mu + aux_term)
                avg_psi = (psi_next + psi_prev_r)/2.0
            
            next_sca_flux_d = avg_psi * weights[d]
            sca_flux_err_d  = np.abs(init_sca_flux_d - next_sca_flux_d)
            init_sca_flux_d = next_sca_flux_d

In [ ]:
aux_term = (sigma_t * dx)/2.0
eps = 1e-14
psi_prev = np.full(((m, ), psi_right))
psi_prev[0::m//2] = psi_left
curr_scalar_fluxes = np.full((ncells, ), np.log(2))
new_scalar_fluxes  = np.zeros(curr_scalar_fluxes.shape)
scalar_flux_err    = np.abs(new_scalar_fluxes - curr_scalar_fluxes)

while np.any(scalar_flux_err >= eps):
    curr_source_term = (q_ext + sigma_s*curr_scalar_fluxes)/2.0
    for d, mu in enumerate(angles):
        abs_mu = np.abs(mu)
        for i in np.arange(ncells):
            if mu > 0:
                psi_d_i_next = ((abs_mu - aux_term) * psi_prev[d] + curr_source_term[i]*dx)/ \
                                (abs_mu + aux_term)
                avg_psi = (psi_d_i_next + psi_d_l)/2.0
            else:
                psi_d_i_next = ((abs_mu - aux_term) * psi_d_r + curr_source_term[i]*dx)/ \
                                (abs_mu + aux_term)
                avg_psi = (psi_d_i_next + psi_d_r)/2.0
            next_sca_flux[i] += avg_psi * weights[d]
    scalar_flux_err = np.abs(new_scalar_fluxes - curr_scalar_fluxes)
    curr_scalar_fluxes = new_scalar_fluxes